In [1]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mncy8Ly8BK2t6tM8aYZEBPRhbwsZD8KxY9
mrckabL9GWjcGytQg1NRvDQat45Umk9mj8
moLdtNAPFZoFN68q5psEg6iHTSezWLvna2
mqiD6KF3CUwX4s6cS2Py4UakqTdhZkhUnT
mu9JroK2CCPQXGtQGynsa8Dd3B7gyrK6Gv
mnRrSqSYmsDAgwmjPiz77HJfz1XsfKM3WW
mix6nFZyiiTwFFhqhQuzUYduhSvYAkMEXN
mwQSGx7tQxPU6ECGBhxPQ4uofsV5RestnZ'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount * 100000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = total - int(fee*100000000) - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
010000000112e497ade6b439d8f7ff9ebd124c964721c632e3168b697936469d28c8689ed8010000006b483045022100a442efd7619acaf0b11d467298adee0d74189e23243bba00771454f5a9598e5702200394e08da906e2cdb2bcb089b761c399210663ef9bf3963cc1e4ba40bb5f16b10121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff0980969800000000001976a9144dec123f6ffd6e37ba81c0d7f2dc0ce28316403388ac80969800000000001976a91479c219af167afa8ef4ee92b711697426dd23aaaa88ac80969800000000001976a91455cd7ae659b93addeae329b421afacf621c8a0ae88ac80969800000000001976a9146fd22f93297b5250b18eaadf4b9d1defe5a7d0d888ac80969800000000001976a9149579c6b8a3d98a1b51cc6fb92524da62f1075b1688ac80969800000000001976a9144bd1eb2457091bf1a4157291a2c11093628f666888ac80969800000000001976a91425a9dc3c576c264ca2bd297e0b2600e4da1ab18488ac80969800000000001976a914ae467549d3c0105a6a4632dd58cc85e9d2d931dd88aca6290992000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000


In [ ]:
tx_in = TxIn(
    bytes.fromhex(
        'e591ea6abf3db4812e6ba594a5015a079bb40b5962414367e210ff5ac0c62826'), 1)
amount = tx_in.value(testnet=True) - 50000
script_pubkey = p2pkh_script(decode_base58('mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'))
print(script_pubkey)
tx_out = TxOut(amount, script_pubkey)

tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
print(tx_obj.serialize().hex())

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_satoshis = total - int((fee)*100000000) - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())